# 행, 열 병합

두 개 이상의 데이터 프레임을 병합하는 처리

- 다른 데이터 프레임과 데이터 합치기 (열)
- 다른 데이터 프레임과 데이터 합치기 (행)

## #01. 준비과정

### [1] 패키지 참조

In [7]:
from pandas import DataFrame, read_excel
from pandas import merge, concat

### [2] 샘플 데이터 가져오기

In [11]:
고객 = read_excel("https://data.hossam.kr/pydata/customer.xlsx")
고객

,고객번호,이름
0,1001,둘리
1,1002,도우너
2,1003,또치
3,1004,길동
4,1005,희동
5,1006,마이콜
6,1007,영희


In [10]:
매출 = read_excel("https://data.hossam.kr/pydata/customer_money.xlsx")
매출

,고객번호,금액
0,1001,10000
1,1001,20000
2,1005,15000
3,1006,5000
4,1008,100000
5,1001,30000


## #02. 다른 데이터 프레임과 데이터 합치기 (열)

### [1] 데이터 프레임 기본 병합

#### (1) 일치하는 데이터끼의 병합
두개의 데이터 프레임에서 이름이 동일한 컬럼을 기준으로 같은 데이터끼리 병합하고 일치하지 않는 데이터는 버려진다.

만약 양쪽 데이터프레임의 공통컬럼에 중복 데이터가 여러개 있는 경우는 모든 경우의 수를 따져서 조합을 만들어 낸다.(예: 1001번 둘리의 데이터)
> SQL의 equi 혹은 inner join과 동일

In [12]:
merge(고객, 매출)

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000


#### (2) 왼쪽 데이터 프레임을 기준으로 병합

고객(왼쪽) 데이터를 기준으로 일치하는 매출 데이터를 병합한다.

고객 데이터의 모든 항목에 대한 출력이 보장된다.

고객 데이터에 매칭되는 매출 데이터가 없는 경우 빈 값으로 처리된다.

> SQL의 LEFT OUTER JOIN과 동일

#### (3) 오른쪽 데이터 프레임을 기준으로 병합


> SQL의 RIGHT OUTER JOIN과 동일

In [13]:
merge(고객, 매출, how='right')

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1005,희동,15000
3,1006,마이콜,5000
4,1008,NaN,100000
5,1001,둘리,30000


#### (4) 모든 데이터의 교차 병합
> SQL의 FULL OUTER JOIN과 동일

In [14]:
merge(고객, 매출, how="outer")

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN
9,1008,NaN,100000.0


### [2] 병합 대상 열 지정하기

#### (1) 샘플 데이터 가져오기

In [15]:
cd1 = read_excel("https://data.hossam.kr/pydata/customer_data1.xlsx")
cd1

,고객명,날짜,데이터
0,민수,2018-01-01,20000
1,수영,2018-01-01,100000


In [16]:
cd2 = read_excel("https://data.hossam.kr/pydata/customer_data2.xlsx")
cd2

,고객명,데이터
0,민수,21세
1,수영,20세


#### (2) 기본 병합

두 데이터 프레임에서 이름이 같은 열은 모두 키가 된다.

샘플 데이터에서는 '데이터'라는 이름의 변수가 `cd1`은 `int`, `cd2`는 `str` 타입이므로 병합 기준을 충족하지 않는다.

그러므로 아래 코드는 에러.

In [17]:
merge(cd1, cd2)

ValueError: You are trying to merge on int64 and object columns for key '데이터'. If you wish to proceed you should use pd.concat

#### (3) 병합 기준 설정
병합 기준 열이 아니면서 이름이 같은 열에는 `_x` 또는 `_y`와 같은 접미사가 붙는다.

In [18]:
tmp = merge(cd1, cd2, on='고객명')
tmp

,고객명,날짜,데이터_x,데이터_y
0,민수,2018-01-01,20000,21세
1,수영,2018-01-01,100000,20세


In [19]:
tmp.rename(columns={'데이터_x': '금액', '데이터_y': '나이'})

,고객명,날짜,금액,나이
0,민수,2018-01-01,20000,21세
1,수영,2018-01-01,100000,20세


#### (4) 두 데이터 프레임의 모든 컬럼 이름이 다른 경우
왼쪽의 기준열 이름과 오른쪽의 기준열 이름을 각각 설정해야 한다.

In [20]:
df_left = DataFrame({'이름': ['영희', '철수'], '국어': [87, 91]})
df_left

,이름,국어
0,영희,87
1,철수,91


In [21]:
df_right = DataFrame({'성명': ['영희', '철수'], '영어': [90, 82]})
df_right

,성명,영어
0,영희,90
1,철수,82


In [22]:
r3 = merge(df_left, df_right, left_on=['이름'], right_on=['성명'])
r3

,이름,국어,성명,영어
0,영희,87,영희,90
1,철수,91,철수,82


In [23]:
r4 = r3.drop('성명', axis=1)    # 이름과 성명은 같으므로, 둘 중 하나는 제거
r4

,이름,국어,영어
0,영희,87,90
1,철수,91,82


### [3] 인덱스를 활용한 병합
#### (1) 인덱스를 기준으로 한 병합
학생의 이름을 인덱스로 갖는 두 데이터 프레임

In [25]:
df_left = DataFrame({'수학': [90, 82]}, index=['민철', '봉구'])
df_left

,수학
민철,90
봉구,82


In [26]:
df_right = DataFrame({'국어': [90, 82]}, index=['민철', '철수'])
df_right

,국어
민철,90
철수,82


병합조건에서 index를 사용하도록 지정(inner join)

In [27]:
merge(df_left, df_right, left_index=True, right_index=True)

,수학,국어
민철,90,90


병합조건에서 index를 사용하도록 지정 (left outer join)

In [28]:
merge(df_left, df_right, left_index=True, right_index=True, how="left")

,수학,국어
민철,90,90.0
봉구,82,NaN


병합조건에서 index를 사용하도록 지정 (right outer join)

In [30]:
merge(df_left, df_right, left_index = True, right_index=True, how="right")

,수학,국어
민철,90.0,90
철수,NaN,82


병합조건에서 index를 사용하도록 지정 (full outer join)

In [31]:
merge(df_left, df_right, left_index= True, right_index=True, how="outer")

,수학,국어
민철,90.0,90.0
봉구,82.0,NaN
철수,NaN,82.0


#### (2) 인덱스와 컬럼을 각각 기준으로 하기

In [32]:
df_left = DataFrame({'수학': [90, 82]}, index=['민철', '봉구'])
df_left

,수학
민철,90
봉구,82


In [33]:
df_right = DataFrame({'성명': ['민철', '철수'], '영어': [90, 82]})
df_right

,성명,영어
0,민철,90
1,철수,82


In [34]:
merge(df_left, df_right, left_index=True, right_on = ['성명'])

,수학,성명,영어
0,90,민철,90


In [35]:
merge(df_left, df_right, left_index=True, right_on=['성명'], how = 'left')

,수학,성명,영어
0.0,90,민철,90.0
NaN,82,봉구,NaN


In [37]:
merge(df_left, df_right, left_index=True, right_on=['성명'], how = 'right')

,수학,성명,영어
0,90.0,민철,90
1,NaN,철수,82


In [38]:
tmp = merge(df_left, df_right, left_index=True, right_on=['성명'], how='outer')
tmp

,수학,성명,영어
0.0,90.0,민철,90.0
NaN,82.0,봉구,NaN
1.0,NaN,철수,82.0


In [39]:
tmp2 = tmp.set_index('성명')
tmp2

,수학,영어
성명,,
민철,90.0,90.0
봉구,82.0,NaN
철수,NaN,82.0


## #03. 다른 데이터 프레임과 데이터 합치기 (행)
### [1] 행 단위 병합 기본 사용 방법

병합할 데이터 프레임을 리스트로 묶는다. (2개 이상 가능)

각 데이터프레임이 갖는 인덱스는 그대로 유지된다.(인덱스 중복 발생)

#### (1) 샘플 데이터 프레임 생성

In [40]:
df1 = DataFrame({'이름': ['영희', '철수'], '국어': [87, 91]})
df1

,이름,국어
0,영희,87
1,철수,91


In [41]:
df2 = DataFrame({'이름': ['민철', '수현'], '국어': [78, 92]})
df2

,이름,국어
0,민철,78
1,수현,92


#### (2) 데이터 프레임 병합

In [42]:
concat([df1, df2])

,이름,국어
0,영희,87
1,철수,91
0,민철,78
1,수현,92


#### (3) 병합 결과에서 index를 재구성하기
지금까지 공부한 내용 활용

In [43]:
kkk = concat([df1, df2])
kkk.reset_index(inplace=True, drop=True)
kkk

,이름,국어
0,영희,87
1,철수,91
2,민철,78
3,수현,92


`concat()` 함수의 파라미터 활용

`ignore_index=True` 파라미터를 설정하면 병합 후 인덱스를 재구성 한다.

In [44]:
concat([df1, df2], ignore_index=True)

,이름,국어
0,영희,87
1,철수,91
2,민철,78
3,수현,92


### [2] 서로 다른 변수를 갖는 데이터 프레임의 병합
#### (1) 샘플 데이터 구성

In [45]:
df3 = DataFrame({'이름': ['영희', '철수'], '국어': [87, 91]})
df3

,이름,국어
0,영희,87
1,철수,91


In [46]:
df4 = DataFrame({'이름': ['민철', '수현'], '수학': [78, 92]})
df4

,이름,수학
0,민철,78
1,수현,92


#### (2) 데이터 병합

일치하지 않는 부분에 대해서는 NaN으로 처리한다

In [47]:
concat([df3, df4], ignore_index=True)

,이름,국어,수학
0,영희,87.0,NaN
1,철수,91.0,NaN
2,민철,NaN,78.0
3,수현,NaN,92.0


### [3] 인덱스를 갖는 데이터 프레임의 병합
#### (1) 샘플 데이터 구성

In [48]:
df5 = DataFrame({'국어': [87, 91, 85]}, index=['영희', '철수', '민수'])
df5

,국어
영희,87
철수,91
민수,85


In [49]:
df6 = DataFrame({'국어': [78, 92, 82]}, index=['민철', '수현', '민수'])
df6

,국어
민철,78
수현,92
민수,82


#### (2) 데이터 병합
인덱스도 데이터 프레임 병합시 함께 병합된다.

단, 각 데이터 프레임간에 중복되는 인덱스가 존재할 경우 각 행은 개별적으로 존재한다

In [51]:
concat([df5, df6])

# 민수가 두 번 반복된다.

,국어
영희,87
철수,91
민수,85
민철,78
수현,92
민수,82


#### (3) 인덱스를 제외하고 병합하기

`ignore_index=True`라는 파라미터를 설정한다.

In [52]:
concat([df5, df6], ignore_index=True)

,국어
0,87
1,91
2,85
3,78
4,92
5,82


## #04. 데이터 프레임 병합 처리 시 참고
### merge() 함수
- 열단위 병합
- 변수, 인덱스 모두 기준으로 설정 가능
- left_on, right_on, left_index, right_index 파라미터가 있다.
- join()보다 사용 범위가 넓다.

### join() 함수
- 열단위 병합
- 인덱스만을 기준으로 설정 가능
- left_on, right_on, left_index, right_index 파라미터가 없다.
- merge()보다 사용 범위가 좁다

### concat() 함수
- axis 파라미터를 사용하면 열단위 병합도 가능

| 메소드                | 설명                                               | 예제                                                | 결과                                      |
|-----------------------|-----------------------------------------------------|------------------------------------------------------|-------------------------------------------|
| `pd.concat()`         | 특정 축을 따라 데이터프레임을 연결합니다.           | `pd.concat([df1, df2], ignore_index=True)`           | 새로운 인덱스로 병합된 데이터프레임.      |
| `pd.concat()`         | 서로 다른 변수를 갖는 데이터프레임을 연결합니다.   | `pd.concat([df3, df4], ignore_index=True)`           | 일치하지 않는 부분은 NaN으로 처리된 데이터프레임. |
| `pd.concat()`         | 특정 축을 따라 데이터프레임을 연결합니다. (열 병합) | `pd.concat([df1, df2], axis=1)`                     | 열을 기준으로 병합된 데이터프레임.         |
| `concat()`            | 특정 축을 따라 데이터프레임을 연결합니다. (행 병합) | `pd.concat([df1, df2], axis=0)`                     | 행을 기준으로 병합된 데이터프레임.         |
| `concat()`            | 특정 열을 기준으로 데이터프레임을 연결합니다.       | `pd.concat([df1, df2], keys=['A', 'B'])`           | 새로운 계층적 인덱스를 생성한 데이터프레임. |
| `merge()`             | 특정 열을 기준으로 데이터프레임을 병합합니다.      | `pd.merge(df_left, df_right, on='key')`            | 'key' 열을 기준으로 병합된 데이터프레임.   |
| `merge()`             | 서로 다른 열 이름을 갖는 데이터프레임을 병합합니다. | `pd.merge(df_left, df_right, left_on='key1', right_on='key2')` | 지정된 열을 기준으로 병합된 데이터프레임. |
| `merge()`             | 인덱스를 기준으로 데이터프레임을 병합합니다.         | `pd.merge(df_left, df_right, left_index=True, right_index=True)` | 인덱스를 기준으로 병합된 데이터프레임.     |
| `join()`              | 인덱스를 기준으로 데이터프레임을 결합합니다.         | `df_left.join(df_right, lsuffix='_left', rsuffix='_right')` | 접미사를 추가하여 결합된 데이터프레임.   |
